# Re-process USA CCS data from Kelly 

Date: July 17, 2025 (Completed: Aug 1, 2025)

Author: Yunha Lee


### Remove the following rows:
 - epa_subpart = -1 : this is the case for a facility is NOT in CO2NCORD but IS in GHGRP. 
 - scc = NaN: this is the case for subparts that are not estimated to be stationary combustion 

### This script will sume NH3 and VOC emissions for multiple ghgrp facilities and/or two subparts C & D. With the file created from this script, the case 2 and 3 duplicates should be removed.

==========

Kelly's USA CCS data has three types of duplicates when sorting by EIS_ID and SCC:

Case 1. multiple sources (most commonly two sources; for example, ptnonipm_2 and ptegu_1)
==>  I plan to split the final CCS emissions into two or more parts, weight by the corresponding NEI emissions.
 
Case 2. multiple ghgrp facilities are linked (usually 2), which has different NH3 and VOC increase emissions; all other species has identical values.
==>  I plan to sum the NH3 and VOC increase emissions across the two (or more) ghgrp facilities.
 
Case 3. Two subparts (C and D), which result in different NH3 and VOC emissions increase (the other emissions are identical).  
==>  I plan to sum the NH3 and VOC increase emissions from both subparts.

==========

Kelly's USA CCS data contains 111 rows that doesn't match back to the NEI base 2020 emissions. For that case, I only put back the CCS's VOC and NH3 emission increase only and set all other emissions, including NEI emissions, to be zero. This way, the NEI emissions in the new CCS emission file matches back to the original NEI emissions.


In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import warnings
import geopandas as gpd
from pyproj import CRS

# Suppress all warnings
warnings.filterwarnings('ignore')

CCS_raw_file = '/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA-compiled-cobenefits-emissions-withSCC-v4.csv'

# This function processes Amy's old CS emission data file.
cs_emis = pd.read_csv(CCS_raw_file, index_col=False)

cs_emis.head()

In [ ]:
## Remove the rows with missing SCC or epa_subpart = -1

print("cs_emis original shape: ", cs_emis.shape)
indices_to_drop = cs_emis[cs_emis['epa_subpart'] == '-1'].index
cs_emis.drop(indices_to_drop, inplace=True)

print("after dropping epa_subpart = -1, cs_emis shape: ", cs_emis.shape)

indices_to_drop = cs_emis[cs_emis['scc'].isna()].index
cs_emis.drop(indices_to_drop, inplace=True)

print("after scc with NaN, cs_emis shape: ", cs_emis.shape)

In [ ]:
# Find all duplicate rows, including the first occurrence
all_duplicates = cs_emis[cs_emis.duplicated(keep=False)]
print("All duplicate rows:")
print(all_duplicates)

# Identify duplicates
duplicate_keys = (
    cs_emis.groupby(['eis_id', 'scc'])
    .size()
    .reset_index(name='count')
    .query('count > 1')[['eis_id', 'scc']]
)
duplicates = cs_emis.merge(duplicate_keys, on=['eis_id', 'scc'], how='inner')
duplicates['row_key'] = duplicates.index  # Track original index

print(duplicates.shape, cs_emis.shape)

# Case 1: Multiple ghgrp_faci for the same NEI (EIS_ID + SCC)
case1_keys = (
    duplicates.groupby(['eis_id', 'scc'])['ghgrp_facility_id']
    .nunique()
    .reset_index(name='ghgrp_faci_count')
    .query('ghgrp_faci_count > 1')[['eis_id', 'scc']]
)
case1 = duplicates.merge(case1_keys, on=['eis_id', 'scc'])
case1_row_keys = set(case1['row_key'])


# Exclude Case 1 rows before doing Case 2
case_others = duplicates[~duplicates['row_key'].isin(case1_row_keys)]

# Case 2: Multiple 'epa_subpart' for the same NEI (EIS_ID + SCC)
case2_keys = (
    case_others.groupby(['eis_id', 'scc'])['epa_subpart']
    .nunique()
    .reset_index(name='epa_subpart_count')
    .query('epa_subpart_count > 1')[['eis_id', 'scc']]
)
case2 = case_others.merge(case2_keys, on=['eis_id', 'scc'])
case2_row_keys = set(case2['row_key'])

print(case1.shape, case2.shape)
duplicates.head()

combined_case_row_keys = case1_row_keys.union(case2_row_keys)
print("Total unique rows in Case 1 or Case 2:", len(combined_case_row_keys))

unexplained = duplicates[~duplicates['row_key'].isin(combined_case_row_keys)]
print("Unexplained duplicates:", unexplained.shape[0])

In [ ]:
# Define species names
species_to_sum = ['VOC', 'NH3']
all_species = ['VOC', 'NH3', 'NOX', 'SO2', 'PM25']

# Manually define sum_cols (from image)
sum_cols = [ 'NH3_increase_SCC_tons','VOC_increase_SCC_tons']

# Dynamically identify all species-related columns, excluding those in sum_cols
CCS_cols = [
    col for col in cs_emis.columns
    if any(sp in col for sp in all_species) and col not in sum_cols
]

# Step 1: Identify duplicate (eis_id, scc) pairs
dup_keys = (
    cs_emis.groupby(['eis_id', 'scc'])
    .size().reset_index(name='count')
    .query('count > 1')[['eis_id', 'scc']]
)

# Step 2: Split duplicated and non-duplicated rows
cs_emis['row_key'] = cs_emis.index
duplicated_rows = cs_emis.merge(dup_keys, on=['eis_id', 'scc'], how='inner')
non_duplicated_rows = cs_emis[~cs_emis['row_key'].isin(duplicated_rows['row_key'])]

# Step 3: Deduplicate only needed rows
grouped = duplicated_rows.groupby(['eis_id', 'scc'])

import numpy as np
def values_consistent(group, col):
    vals = group[col].dropna().unique()
    return len(vals) <= 1

exclude_consistency_check = sum_cols + ['NH3_increase_subpart_tons', 'VOC_increase_subpart_tons', 'frac_of_VOC_subpart']

# Compare only the columns where values differ between the two rows
def get_diff_columns(group):
    if group.shape[0] != 2:
        raise ValueError("Expected group with exactly 2 rows")
    
    # Compare the two rows
    diffs = (group.iloc[0] != group.iloc[1]) & ~(group.iloc[0].isna() & group.iloc[1].isna())
    
    # Return only the differing columns
    return group.loc[:, diffs]

dedup_list = []
for key, group in grouped:
    ref = group.iloc[0]
    consistent = all(values_consistent(group, col) for col in CCS_cols if col not in exclude_consistency_check)
    if not consistent:
        print(f"❌ Inconsistent values for group: {key}")
        print(get_diff_columns(group[CCS_cols]))
        raise ValueError(f"Inconsistent species values in group: {key}")
    
    summed_row = ref.copy()
    for col in sum_cols:
        summed_row[col] = group[col].sum()
    dedup_list.append(summed_row)

dedup_df = pd.DataFrame(dedup_list)

# Step 4: Final dataframe
cs_emis_final = pd.concat([non_duplicated_rows, dedup_df], ignore_index=True)
cs_emis_final.drop(columns='row_key', inplace=True)

# Step 5: Sanity check
print("\n==== 🔍 CCS_cols Sum Check ====")
before = cs_emis[CCS_cols].sum()
after = cs_emis_final[CCS_cols].sum()
diff = after - before

check_df = pd.DataFrame({
    'Before': before,
    'After': after,
    'Diff': diff
})
print(check_df)

# Check row counts
print(f"\nOriginal rows: {cs_emis.shape[0]}, Final rows: {cs_emis_final.shape[0]}")
print(f"Duplicates handled: {duplicated_rows.shape[0] - dedup_df.shape[0]}")


In [ ]:
print(cs_emis.shape, cs_emis_final.shape)

In [ ]:
## To verify the sum of NOx, SO2 and PM2.5, the sum from the "non_duplicated_rows" and the sum from the first of "duplicated_rows" should be compared to the sum to the cs_emis_final. 

# Step 1: Sum from non-duplicated rows
non_dup_sum = non_duplicated_rows[CCS_cols].sum()

# Step 2: For duplicated groups, get the first row only
first_of_duplicates = duplicated_rows.groupby(['eis_id', 'scc'], as_index=False).first()
first_dup_sum = first_of_duplicates[CCS_cols].sum()

# Step 3: Compare to full final deduplicated dataframe
final_sum = cs_emis_final[CCS_cols].sum()

# Step 4: Combine and compare
verify_df = pd.DataFrame({
    'Non-Duplicated': non_dup_sum,
    'First of Duplicates': first_dup_sum,
    'Reconstructed (NonDup + FirstDup)': non_dup_sum + first_dup_sum,
    'Final Deduplicated': final_sum,
    'Diff': final_sum - (non_dup_sum + first_dup_sum)
})

print("\n=== 🔍 Emission Verification ===")
print(verify_df)


In [ ]:
CCS_missing_species = ['VOC', 'NH3']

for missing in CCS_missing_species:
    # sometimes, kelley's output missing these species output
    missing_name = missing + "_out_subpart_tons"
    if missing_name not in cs_emis.columns:
        print("computing missing species output :", missing_name)
       
        cs_emis[missing_name] = cs_emis[missing + '_subpart_tons'].fillna(0)  + cs_emis[missing + '_increase_SCC_tons']

In [ ]:
# Drop unnecessary columns for my emission processing
columns_to_drop = [col for col in cs_emis_final.columns if "CO2" in col]
columns_to_drop = columns_to_drop + [col for col in cs_emis_final.columns if "cost_" in col]

cs_emis_final.drop(columns=columns_to_drop, inplace=True)
print(cs_emis_final.columns)

In [ ]:
cs_emis_final.to_csv("/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA-compiled-cobenefits-emissions-withSCC-v4_ghgrp_subpart_sum.csv", index=False )

In [ ]:
# some rows missing scc
missing_scc = cs_emis_final[cs_emis_final['epa_subpart'] == "-1"]

missing_scc_sum = missing_scc[CCS_cols].sum()
print(missing_scc_sum)

print(cs_emis_final[CCS_cols].sum())

## Apply CCS emission to NEI

## Step 1: Read NEI and CCS original data file

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import warnings
import geopandas as gpd
from pyproj import CRS

# Suppress all warnings
warnings.filterwarnings('ignore')

# Add the path to the main package directory
package_path = os.path.abspath('/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/LOCAETA_AQ')
if package_path not in sys.path:
    sys.path.append(package_path)

import emission_processing

# output file path for processed emissions 
NEI_CCS_emis_file = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA_CCS_combined_NEI_point_oilgas_ptegu_ptnonimps.shp"
State_emis_file = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA_point_CCS.shp"
State_CCS_emis_file = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA_point_CCS_reduced_emis.shp"
output_plots_dir ='/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/'

# CCS and NEI raw data directory
nei_pt_path = '/Users/yunhalee/Documents/LOCAETA/RCM/INMAP/evaldata_v1.6.1/2020_nei_emissions/'
combined_NEI_emis_path = nei_pt_path +'combined_NEI2020_pt_oilgas_ptegu_ptnonipm_w_sectors.shp'

target_crs = "+proj=lcc +lat_1=33.000000 +lat_2=45.000000 +lat_0=40.000000 +lon_0=-97.000000 +x_0=0 +y_0=0 +a=6370997.000000 +b=6370997.000000 +to_meter=1"

gdf = gpd.read_file(combined_NEI_emis_path)



In [ ]:
CCS_raw_file = '/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA-compiled-cobenefits-emissions-withSCC-v4_ghgrp_subpart_sum.csv' #'/Users/yunhalee/Documents/LOCAETA/CS_emissions/final_output_1_manual_update_noLandfill.csv'

# process CCS emissions to merge it with NEI emissions
cs_emis = pd.read_csv(CCS_raw_file)

# ensure scc column to be integer
cs_emis['scc'] = cs_emis['scc'].astype(int)

cs_emis.rename(columns={'eis_id': 'EIS_ID', 'scc': 'SCC'}, inplace = True)

cs_emis.head()

if 'PM25_reduction_subpart_tons' not in cs_emis.columns:
    print('PM25_reduction_subpart_tons is missing, so now computed')
    cs_emis['PM25_reduction_subpart_tons'] = cs_emis['PM25CON_reduction_subpart_tons'] + cs_emis['PM25FIL_reduction_subpart_tons']

## Pre-step 2: Check if the subpart_tons matches NEI emissions

In [ ]:
# STEP 0: Define key columns and make a copy
NEI_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']
CCS_subpart_cols = ['VOC_subpart_tons', 'NOX_subpart_tons', 'NH3_subpart_tons','SO2_subpart_tons', 'PM25_subpart_tons']
CCS_changes_cols = ['VOC_increase_SCC_tons', 'NOX_reduction_subpart_tons', 'NH3_increase_SCC_tons','SO2_reduction_subpart_tons', 'PM25_reduction_subpart_tons'] 

key_cols = ['EIS_ID', 'SCC']
gdf_copy = gdf.copy()

# STEP 1: Aggregate NEI values by (EIS_ID, SCC)
gdf_agg = gdf_copy.groupby(key_cols)[NEI_cols].sum().reset_index()

# STEP 2: Merge with cs_emis for comparison
compare_df = pd.merge(
    gdf_agg,
    cs_emis[key_cols + CCS_subpart_cols + CCS_changes_cols],
    on=key_cols,
    how='inner'
)

import numpy as np

# STEP 3: Compare each NEI vs CCS column
for nei_col, ccs_col in zip(NEI_cols, CCS_subpart_cols):
    nei_vals = compare_df[nei_col]
    ccs_vals = compare_df[ccs_col]
    
    match_mask = (
        np.isclose(nei_vals, ccs_vals, equal_nan=True) |
        ((nei_vals.isna() & (ccs_vals == 0)) | ((nei_vals == 0) & ccs_vals.isna()))
    )

    mismatch_mask = ~match_mask

    if mismatch_mask.any():
        print(f"\n❌ Mismatched values for {nei_col} vs {ccs_col}:")
        print(compare_df.loc[mismatch_mask, key_cols + [nei_col, ccs_col]])
        print(compare_df.loc[mismatch_mask, [nei_col, ccs_col]].sum())

# Compare the net difference (between NEI and CCS subpart) to the changes of CCS emissions. 
for chg_col, ccs_col in zip(CCS_changes_cols, CCS_subpart_cols):
    print(compare_df.loc[mismatch_mask, [chg_col, ccs_col]].sum())

## Important note: There is very small difference in CCS subpart emissions and NEI emissions, likely due to the difference between NEI raw data and the NEI SMOKE formatted data. This might be a bug in NEI SMOKE, but I can't fix this directly. So I am going to apply the changes of CCS emissions into my NEI emissions (from NEI SMOKE formatted data) to avoid this inconsistency issue. 

## Step 2: Find the matching group of EIS_ID and SCC between cs_emis and NEI and allocate the cs_emis into the NEI emissions dataframe

In [ ]:
# STEP 0: Define key columns and make a copy
NEI_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']
CCS_cols = ['VOC_increase_SCC_tons', 'NOX_reduction_subpart_tons',  'NH3_increase_SCC_tons', 'SO2_reduction_subpart_tons', 'PM25_reduction_subpart_tons'] 

#CCS_cols = ['VOC_out_subpart_tons', 'NH3_out_subpart_tons', 'NOX_out_subpart_tons','SO2_out_subpart_tons', 'PM25_out_subpart_tons']

key_cols = ['EIS_ID', 'SCC']
gdf_copy = gdf.copy()

# STEP 1: Identify matched (EIS_ID, SCC) pairs
matched_keys = pd.merge(
    gdf_copy[key_cols].drop_duplicates(),
    cs_emis[key_cols].drop_duplicates(),
    on=key_cols
)

# STEP 2: Split gdf into matched and unmatched
gdf_matched = gdf_copy.merge(matched_keys, on=key_cols, how='inner')
gdf_unmatched = gdf_copy.merge(matched_keys, on=key_cols, how='outer', indicator=True).query('_merge == "left_only"').drop(columns='_merge')

# STEP 3: Merge CCS values into matched gdf
gdf_matched = gdf_matched.merge(cs_emis[key_cols + CCS_cols], on=key_cols, how='left', suffixes=('', '_ccs'))

# STEP 4: Allocation only for matched rows
def allocate(group):
    n = len(group)
    for nei_col, ccs_col in zip(NEI_cols, CCS_cols):
        ccs_val = group[f"{ccs_col}"].iloc[0]

        if pd.isna(ccs_val):
            group[ccs_col] = np.nan
            continue

        nei_vals = group[nei_col]
        total_nei = nei_vals.sum()

        if pd.isna(total_nei):
            group[ccs_col] = np.nan
        elif total_nei > 0:
            group[ccs_col] = (nei_vals / total_nei) * ccs_val
        else:
            group[ccs_col] = ccs_val / n
    return group

print("=== Allocating CCS only for matched groups ===")
gdf_matched_allocated = gdf_matched.groupby(key_cols, group_keys=False).apply(allocate)

# Drop helper columns (e.g., *_ccs) if needed
gdf_matched_allocated = gdf_matched_allocated.drop(columns=[f"{c}_ccs" for c in CCS_cols], errors='ignore')

# STEP 5: Combine with unmatched (with CCS columns as NaN)
for c in CCS_cols:
    if c not in gdf_unmatched.columns:
        gdf_unmatched[c] = np.nan

final = pd.concat([gdf_matched_allocated, gdf_unmatched], ignore_index=True)

# STEP 6: conservation check for the allocated vs matched original)
print("\n=== Conservation Check ===")
cs_allocated = final.groupby(key_cols)[CCS_cols].sum().reset_index()
cs_original = cs_emis[key_cols + CCS_cols]
check = cs_allocated.merge(cs_original, on=key_cols, suffixes=('_alloc', '_orig'))

for col in CCS_cols:
    rel_diff = abs(check[f"{col}_alloc"] - check[f"{col}_orig"]) / (check[f"{col}_orig"])
    if (rel_diff > 0.00001).any():
        print(f"[Conservation Issue] {col}: max diff = {rel_diff.max()*100:.4f}%")


print("=== The number of rows should be matched ===", gdf.shape, gdf_matched_allocated.shape, gdf_unmatched.shape)
print("=== The number of rows should be matched ===", check.shape, matched_keys.shape)

# STEP 7: Conservation check for the original and final dataframe (this will be different, if there is unmatched case)
total_sum_original = cs_emis[CCS_cols].sum()
total_sum_allocated = final[CCS_cols].sum()

print("\n=== FINAL CONSERVATION CHECK ===")
print("Original Total CCS change Emissions:")
print(total_sum_original)
print("Allocated Total CCS change Emissions:")
print(total_sum_allocated)
print("Relative Difference:")
rel_diff = (total_sum_original- total_sum_allocated) / (total_sum_original)
print(rel_diff)

if any(abs(rel_diff) > 0.0001):
    print("❌ CONSERVATION FAILED!")
else:
    print("✅ CONSERVATION PASSED!")

# check if the rows of is_nan matches with allocated cs_emis
print("=== The number of rows should be matched ===", gdf.shape, final.shape, gdf_matched_allocated.shape[0] + gdf_unmatched.shape[0])

## Step 3: Assign the unmatched CCS emissions back to the final dataframe. I reset all other emissions to zero except NH3 and VOC CCS emission increase. 

In [ ]:
# STEP 8: Add unmatched cs_emis groups into final

# 1. Identify unmatched (EIS_ID, SCC) in cs_emis that are not in final
final_keys = final[['EIS_ID', 'SCC']].drop_duplicates()
unmatched_cs_emis = cs_emis.merge(final_keys, on=['EIS_ID', 'SCC'], how='outer', indicator=True).query('_merge == "left_only"').drop(columns='_merge')

# 2. For each unmatched row, try to get the columns with facility info from final based on EIS_ID (I confirmed that all EIS_ID in the unmatched case are existed in NEI)
rest_cols = [col for col in final.columns if col not in CCS_cols + ['SCC', 'EIS_ID']]
unmatched_lookup = final.drop_duplicates('EIS_ID')[['EIS_ID'] + rest_cols].set_index('EIS_ID')

# Merge the facility columns by EIS_ID
unmatched_with_rest = unmatched_cs_emis.merge(unmatched_lookup, on='EIS_ID', how='left')

NEI_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']
CCS_cols = ['VOC_increase_SCC_tons', 'NOX_reduction_subpart_tons',  'NH3_increase_SCC_tons', 'SO2_reduction_subpart_tons', 'PM25_reduction_subpart_tons'] 
# 3. Set all NEI emissions and NOx, SO2, and PM2.5 CCS emissions to zero. For VOC and NH3 CCS emissions, ensure they are present
for col in CCS_cols:
    if col in ['NOX_reduction_subpart_tons', 'SO2_reduction_subpart_tons', 'PM25_reduction_subpart_tons']:
        unmatched_with_rest[col] = 0.0

for col in NEI_cols:
    unmatched_with_rest[col] = 0.0

# 4. Keep only columns matching final and reorder
final_cols = final.columns
unmatched_with_rest = unmatched_with_rest[[col for col in final_cols if col in unmatched_with_rest.columns]]

print("before adding unmatched_with_rest", final[CCS_cols].sum(), final.shape)
# 5. Append to final
final2 = pd.concat([final, unmatched_with_rest], ignore_index=True)

print("after adding unmatched_with_rest", final2[CCS_cols].sum(), final2.shape)

if final2.shape[0] == gdf_matched_allocated.shape[0] + gdf_unmatched.shape[0]+ unmatched_cs_emis.shape[0]:
    print ("Grat! final # of row are confirmed to be good")

else:
    print("Bad! something not righ about final # of row using matched and unmatched", final2.shape[0],  gdf_matched_allocated.shape[0] + gdf_unmatched.shape[0]+ unmatched_cs_emis.shape[0], gdf_matched_allocated.shape[0],  gdf_unmatched.shape[0], unmatched_cs_emis.shape[0])


#6 check_conservation between original and final (Since PM2.5, SO2, and NOx unmatched cases are set to zero, abs_diff should catch them.)
total_sum_original = cs_emis[CCS_cols].sum()
total_sum_allocated = final2[CCS_cols].sum()

print("\n=== FINAL CONSERVATION CHECK ===")
print("Original Total CCS Emissions:")
print(total_sum_original)
print("Allocated Total CCS Emissions:")
print(total_sum_allocated)
print("Absoluate Difference:")
abs_diff = (total_sum_original- total_sum_allocated) 
print(abs_diff)

    
# Check if the difference between original and final comes from the "unmatched" case (If not, something is wrong)
unmatched_sum = unmatched_cs_emis[CCS_cols].sum()
for sp in CCS_cols:
    if (unmatched_sum [sp] / rel_diff[sp]) > 0.999 :
        print("✅ CONSERVATION PASSED!", unmatched_sum [sp], abs_diff[sp] )
    else:
        print("CONSERVATION FAIL!", unmatched_sum [sp], abs_diff[sp])


## Step 4 - Compute the final CCS emissions based on the changed of CCS emissions

In [ ]:
NEI_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']
CCS_cols = ['VOC_increase_SCC_tons', 'NOX_reduction_subpart_tons',  'NH3_increase_SCC_tons', 'SO2_reduction_subpart_tons', 'PM25_reduction_subpart_tons'] 

# Step 1: Identify rows where all CCS columns are NaN
all_ccs_missing = final2[CCS_cols].isna().all(axis=1) # gdf_match case contains 42 NaN rows due to SCC allocation: gdf_matched[gdf_matched[CCS_cols].isna().all(axis=1)]
all_ccs_missing_df = final2[all_ccs_missing]

# Save original NEI columns as nei and compute new CCS emissions 
for nei_col, ccs_col in zip(NEI_cols, CCS_cols):
    final2[f'{nei_col}_nei'] = final2[nei_col]

    if nei_col in ['NH3', 'VOC']: 
        final2[nei_col] = final2[f'{nei_col}_nei'] +  final2[ccs_col].fillna(0) 
    else:
        final2[nei_col] = final2[f'{nei_col}_nei'] -  final2[ccs_col].fillna(0)

    # Optional: debug first few affected rows
    if all_ccs_missing_df[nei_col].sum() > 0:
        print(final2.loc[all_ccs_missing, [nei_col, ccs_col]].head())

if final2.shape[0] == gdf.shape[0] + unmatched_cs_emis.shape[0]:
    print ("Great! final2 # of row are summed to be gdf # of row and unmatched cs_emis")
else:
    print("Bad! something not right about final2 # of row using gdf", final2.shape[0], gdf.shape[0],  unmatched_cs_emis.shape[0])

if all_ccs_missing.sum() == gdf_unmatched.shape[0]:
    print ("Great! missing rows are matched with gdf_unmatched case!")


NEI_cols = ['VOC_nei', 'NOx_nei', 'NH3_nei', 'SOx_nei', 'PM2_5_nei']
CCS_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']  

# Compute total difference for each pollutant
diff_dict = {}
for nei_col, ccs_col in zip(NEI_cols, CCS_cols):
    nei_total = final2[nei_col].sum()
    ccs_total = final2[ccs_col].sum()
    diff_dict[ccs_col] = ccs_total - nei_total

print(diff_dict)

In [ ]:
CCS_cols = ['VOC_increase_SCC_tons', 'NOX_reduction_subpart_tons',  'NH3_increase_SCC_tons', 'SO2_reduction_subpart_tons', 'PM25_reduction_subpart_tons'] 
final2[CCS_cols].sum()

## Save the final data to the INMAP emission file format

In [ ]:
CCS_cols = ['VOC_increase_SCC_tons', 'NOX_reduction_subpart_tons',  'NH3_increase_SCC_tons', 'SO2_reduction_subpart_tons', 'PM25_reduction_subpart_tons'] 
final2.drop(CCS_cols, axis=1, inplace=True)

filename = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/whole_USA_CCS.shp"
final2.to_file(filename, driver='ESRI Shapefile')
print(f"Saved final data to {filename}")

#reset VOC and NH3 emissions with NEI (no increase)
for sp in ['VOC', 'NH3']:
    final2[sp+'_ccs'] = final2[sp] 
    final2[sp] = final2[sp+'_nei'] 

final2[NEI_cols].sum()

filename = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/whole_USA_CCS_wo_NH3_VOC.shp"
final2.to_file(filename, driver='ESRI Shapefile')
print(f"Saved final data to {filename}")

# Plot the sum

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Define file paths
file_with_ccs = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/whole_USA_CCS.shp"

# Load shapefiles
gdf_with_ccs = gpd.read_file(file_with_ccs)
gdf_with_ccs.head()

In [ ]:
NEI_cols = ['VOC_nei', 'NOx_nei', 'NH3_nei', 'SOx_nei', 'PM2_5_nei']
CCS_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']  

# Compute total difference for each pollutant
diff_dict = {}
for nei_col, ccs_col in zip(NEI_cols, CCS_cols):
    nei_total = gdf_with_ccs[nei_col].sum()
    ccs_total = gdf_with_ccs[ccs_col].sum()
    diff_dict[ccs_col] = ccs_total - nei_total

print(diff_dict)

# Convert to Series for plotting
diff_series = pd.Series(diff_dict)

# Plotting
plt.figure(figsize=(10, 6))
bars = plt.bar(diff_series.index, diff_series.values)
plt.ylabel("Difference in Total Emissions (CCS - NEI) [tons]")
plt.title("Difference Between NEI and CCS Emissions by Pollutant")
plt.axhline(0, color='black', linestyle='--')
# Add value labels on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f'{height:,.2f}',
        ha='center',
        va='bottom' if height >= 0 else 'top'
    )

plt.tight_layout()
plt.show()

In [ ]:
print("these two emissions sum must be same")
print(gdf_with_ccs[NEI_cols].sum())
print(gdf[CCS_cols].sum())

print("New CCS emissions sum are the followings:")
print(gdf_with_ccs[CCS_cols].sum())


print("111 difference comes from unmatched CCS emissions, which I reset all emissions to be zero except NH3 and VOC CCS increase emissions")
print(gdf_with_ccs.shape, gdf.shape, gdf_with_ccs.shape[0]- gdf.shape[0])


## Re-create LA_CCS emissions

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Define file paths
file_with_ccs = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA_CCS.shp"

# Load shapefiles
gdf_with_ccs = gpd.read_file(file_with_ccs)
gdf_with_ccs.head()

In [ ]:
# Define columns
NEI_cols = ['VOC_nei', 'NOx_nei', 'NH3_nei', 'SOx_nei', 'PM2_5_nei']
CCS_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']

# Identify Colorado rows using FIPS (Colorado FIPS starts with '08')
is_colorado = gdf_with_ccs['FIPS'].str.startswith('22')

print("after subsetting", gdf_with_ccs[CCS_cols].sum())

# Split into Colorado and other states
gdf_STATE = gdf_with_ccs[is_colorado].copy()
gdf_other = gdf_with_ccs[~is_colorado].copy()

print("after subsetting", gdf_STATE[CCS_cols].sum()+ gdf_other[CCS_cols].sum())

# For other states: backup original CCS columns, then replace with NEI columns
for nei_col, ccs_col in zip(NEI_cols, CCS_cols):
    gdf_other[f"{ccs_col}_ccs"] = gdf_other[ccs_col]      # Backup original CCS
    gdf_other[ccs_col] = gdf_other[nei_col]              # Overwrite with NEI

# Save to file
gdf_other.to_file("/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA_CCS_without_LA.shp", driver='ESRI Shapefile')
gdf_STATE.to_file("/Users/yunhalee/Documents/LOCAETA/CS_emissions/LA_CCS.shp", driver='ESRI Shapefile')

print("before reset NH3 and VOC", gdf_STATE[CCS_cols].sum())

#reset VOC and NH3 emissions with NEI (no increase)
for sp in ['VOC', 'NH3']:
    gdf_STATE[sp+'_ccs'] = gdf_STATE[sp] 
    gdf_STATE[sp] = gdf_STATE[sp+'_nei'] 

print("after reset NH3 and VOC", gdf_STATE[CCS_cols].sum())

filename = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/LA_CCS_wo_NH3_VOC.shp"
gdf_STATE.to_file(filename, driver='ESRI Shapefile')


In [ ]:
# Read the subset CCS and plot the emissions
filename = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/LA_CCS.shp"
gdf_ccs = gpd.read_file(filename)
gdf_ccs.head()

In [ ]:
import pandas as pd

NEI_cols = ['VOC_nei', 'NOx_nei', 'NH3_nei', 'SOx_nei', 'PM2_5_nei']
CCS_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']  

# Compute total difference for each pollutant
diff_dict = {}
for nei_col, ccs_col in zip(NEI_cols, CCS_cols):
    nei_total = gdf_ccs[nei_col].sum()
    ccs_total = gdf_ccs[ccs_col].sum()
    diff_dict[ccs_col] = ccs_total - nei_total

print(diff_dict)

# Convert to Series for plotting
diff_series = pd.Series(diff_dict)

# Plotting
plt.figure(figsize=(10, 6))
bars = plt.bar(diff_series.index, diff_series.values)
plt.ylabel("Difference in Total Emissions (CCS - NEI) [tons]")
plt.title("Difference Between NEI and CCS Emissions by Pollutant")
plt.axhline(0, color='black', linestyle='--')
# Add value labels on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f'{height:,.2f}',
        ha='center',
        va='bottom' if height >= 0 else 'top'
    )

plt.tight_layout()
plt.show()

## Re-create CO_CCS emissions with Kelly's new data

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Define file paths
file_with_ccs = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA_CCS.shp"

# Load shapefiles
gdf_with_ccs = gpd.read_file(file_with_ccs)
gdf_with_ccs.head()

In [ ]:
import pandas as pd

# Read CO_CCS old data to get EIS_ID and SCC for the facilities we need
CO_CCS_raw_file = '/Users/yunhalee/Documents/LOCAETA/CS_emissions/final_output_1_manual_update_noLandfill.csv'

# process CCS emissions to merge it with NEI emissions
co_ccs_old = pd.read_csv(CO_CCS_raw_file)

# drop if scc is missing
indices_to_drop = co_ccs_old[co_ccs_old['scc'].isna()].index
co_ccs_old.drop(indices_to_drop, inplace=True)

# drop frs columns (because it introduces duplicates)
co_ccs_old.drop(co_ccs_old.filter(regex='frs').columns, axis=1, inplace=True)
co_ccs_old.rename(columns={'eis_id': 'EIS_ID', 'scc': 'SCC'}, inplace = True)


In [ ]:
# Find all duplicate rows, including the first occurrence
all_duplicates = co_ccs_old[co_ccs_old.duplicated(keep=False)]
print("All duplicate rows:")
print(all_duplicates)

# Identify duplicates
duplicate_keys = (
    co_ccs_old.groupby(['EIS_ID', 'SCC'])
    .size()
    .reset_index(name='count')
    .query('count > 1')[['EIS_ID', 'SCC']]
)
duplicates = co_ccs_old.merge(duplicate_keys, on=['EIS_ID', 'SCC'], how='inner')
duplicates['row_key'] = duplicates.index  # Track original index

print(duplicates.shape, co_ccs_old.shape)

In [ ]:
# drop any duplicated rows
co_ccs_old = co_ccs_old.drop_duplicates()

if co_ccs_old.duplicated(subset = ['EIS_ID','SCC']).any():
    print("Warning: co_ccs_old contains duplicate (EIS_ID, SCC) pairs.")
else:
    print("All (EIS_ID, SCC) pairs in co_ccs_old are unique.")

# Step 2: Subset gdf_with_ccs using those pairs
key_cols = ['EIS_ID', 'SCC']

# Merge to get matching rows
gdf_subset = gdf_with_ccs.merge(co_ccs_old[key_cols], on=key_cols, how='inner')
gdf_rest = gdf_with_ccs.merge(co_ccs_old[key_cols], on=key_cols, how='outer', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])

# Optional sanity check
assert len(gdf_subset) + len(gdf_rest) == len(gdf_with_ccs), "❌ Row count mismatch!"

# Preview
print(f"✅ Subset rows: {len(gdf_subset)}")
print(f"✅ Remaining rows: {len(gdf_rest)}")

In [ ]:
# check if there any duplicates in the subset
if gdf_subset.duplicated(subset = ['EIS_ID', 'SCC', 'rel_point_', 'source_fil']).any() == True:
    print ("there is duplicated, need an attention")
else:
    print("all is good")

In [ ]:
# Define columns
NEI_cols = ['VOC_nei', 'NOx_nei', 'NH3_nei', 'SOx_nei', 'PM2_5_nei']
CCS_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']

print("before subsetting", gdf_with_ccs[CCS_cols].sum())

print("after subsetting", gdf_subset[CCS_cols].sum() + gdf_rest[CCS_cols].sum())

# For other states: backup original CCS columns, then replace with NEI columns
for nei_col, ccs_col in zip(NEI_cols, CCS_cols):
    gdf_rest[f"{ccs_col}_ccs"] = gdf_rest[ccs_col]      # Backup original CCS
    gdf_rest[ccs_col] = gdf_rest[nei_col]              # Overwrite with NEI

# Save to file
gdf_rest.to_file("/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA_CCS_without_CO_CCS.shp", driver='ESRI Shapefile')
gdf_subset.to_file("/Users/yunhalee/Documents/LOCAETA/CS_emissions/CO_CCS.shp", driver='ESRI Shapefile')

print("before reset NH3 and VOC", gdf_subset[CCS_cols].sum())

#reset VOC and NH3 emissions with NEI (no increase)
for sp in ['VOC', 'NH3']:
    gdf_subset[sp+'_ccs'] = gdf_subset[sp] 
    gdf_subset[sp] = gdf_subset[sp+'_nei'] 

print("after reset NH3 and VOC", gdf_subset[CCS_cols].sum())

filename = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/CO_CCS_wo_NH3_VOC.shp"
gdf_subset.to_file(filename, driver='ESRI Shapefile')


In [ ]:
import pandas as pd

# Read the subset CCS and plot the emissions
filename = "/Users/yunhalee/Documents/LOCAETA/CS_emissions/CO_CCS.shp"
gdf_co_ccs = gpd.read_file(filename)
gdf_co_ccs.head()

NEI_cols = ['VOC_nei', 'NOx_nei', 'NH3_nei', 'SOx_nei', 'PM2_5_nei']
CCS_cols = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']  

# Compute total difference for each pollutant
diff_dict = {}
for nei_col, ccs_col in zip(NEI_cols, CCS_cols):
    nei_total = gdf_co_ccs[nei_col].sum()
    ccs_total = gdf_co_ccs[ccs_col].sum()
    diff_dict[ccs_col] = ccs_total - nei_total

print(diff_dict)

# Convert to Series for plotting
diff_series = pd.Series(diff_dict)

# Plotting
plt.figure(figsize=(10, 6))
bars = plt.bar(diff_series.index, diff_series.values)
plt.ylabel("Difference in Total Emissions (CCS - NEI) [tons]")
plt.title("Difference Between NEI and CCS Emissions by Pollutant")
plt.axhline(0, color='black', linestyle='--')
# Add value labels on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f'{height:,.2f}',
        ha='center',
        va='bottom' if height >= 0 else 'top'
    )

plt.tight_layout()
plt.show()

In [ ]:
# checking if the subset includes total 9 facilities
if len(gdf_subset['EIS_ID'].unique()) == 9:
    print("All facilities are included")
else:
    print("Something is not right")